In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DateType, TimestampType, FloatType
from pyspark.sql.functions import current_timestamp, sum, quarter, month, date_format, current_date
from pyspark.sql.functions import col, lit, when

spark = SparkSession.builder \
    .appName("Retails sales") \
    .getOrCreate()

dbutils.secrets.listScopes()
dbutils.secrets.list('secretscope')
service_credential = dbutils.secrets.get(scope="secretscope", key="secretid")

spark.conf.set("fs.azure.account.auth.type.salesprojectstorage.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.salesprojectstorage.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.salesprojectstorage.dfs.core.windows.net", "cfdf2f4b-0dd6-4003-85c6-42f61c6e1fe7")
spark.conf.set("fs.azure.account.oauth2.client.secret.salesprojectstorage.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.salesprojectstorage.dfs.core.windows.net", "https://login.microsoftonline.com/80d8b5a9-a2e9-4997-a38b-d0e9c44ef5d6/oauth2/token")

schema = StructType([
    StructField('path', StringType()),
    StructField('name', StringType()),
    StructField('size', IntegerType()),
    StructField('modificationTime', LongType())
])

directory_path = "abfs://silver@salesprojectstorage.dfs.core.windows.net"
file_list = dbutils.fs.ls(directory_path)

df = spark.createDataFrame(data=file_list, schema=schema)

df = df.select('path', 'name', 'size', (df.modificationTime/1000).cast('timestamp').alias('UpdatedTime'))
df_latestfile = df.select('path', 'name', 'size', 'UpdatedTime').orderBy(df.UpdatedTime.desc()).limit(1)

# Extract the file path from df_latestfile
file_path = df_latestfile.select('path').collect()[0][0]

Schema = StructType([
    StructField("Order_ID", StringType()),
    StructField("Order_Date", DateType()),
    StructField("CustomerName", StringType()),
    StructField("State", StringType()),
    StructField("City", StringType()),
    StructField("Order ID", StringType()),
    StructField("Amount", IntegerType()),
    StructField("Profit", IntegerType()),
    StructField("Quantity", IntegerType()),
    StructField("Category", StringType()),
    StructField("Sub-Category", StringType()),
    StructField("PaymentMode", StringType()),
])

df_readingfile = spark.read.csv(file_path, header=True, schema=Schema)
df_readingfile = df_readingfile.drop("Order ID")
df_readingfile = df_readingfile.withColumn("Timestamp", current_timestamp()) \
                               .withColumn("quarter", quarter(df_readingfile['Order_Date']))\
                               .withColumn("insert_timestamp", date_format(col("Timestamp"), "yyyy-MM-dd HH:mm:ss"))

df_profit_Bymonth = df_readingfile.groupBy(date_format(df_readingfile['Order_Date'],"MMMM").alias("month")).agg(sum("Profit").alias("Total profit"))
df_profit_Bymonth.show()

df_totalamount_Bystate = df_readingfile.groupBy("State").agg(sum("Amount").alias("Total Amount"))

df_totalamount_Bystate.show()

df_totalquantity_paymentmode = df_readingfile.groupBy("PaymentMode").agg(sum("Quantity").alias("Total Quantity"))



df_readingfile.write.mode("append").parquet("abfs://gold@salesprojectstorage.dfs.core.windows.net/transformedfile")
df_profit_Bymonth.write.mode("append").parquet("abfs://gold@salesprojectstorage.dfs.core.windows.net/profitBymonthtransformedfile")
df_totalamount_Bystate.write.mode("append").parquet("abfs://gold@salesprojectstorage.dfs.core.windows.net/totalAmountByStatetransformedfile")
df_totalquantity_paymentmode.write.mode("append").parquet("abfs://gold@salesprojectstorage.dfs.core.windows.net/totalquantityBypaymentmodetransformedfile")


+---------+------------+
|    month|Total profit|
+---------+------------+
|     July|       -2138|
| November|       10253|
| February|        8465|
|  January|        9684|
|    March|        7793|
|  October|        2959|
|      May|       -3730|
|   August|        2068|
|    April|        4192|
|     June|         420|
| December|       -1604|
|September|       -1399|
+---------+------------+

+-----------------+------------+
|            State|Total Amount|
+-----------------+------------+
|         Nagaland|       11993|
|        Karnataka|       12520|
|       Tamil Nadu|        6276|
|   Andhra Pradesh|       13256|
|   Madhya Pradesh|       87463|
|           Punjab|       16786|
|              Goa|        6705|
| Himachal Pradesh|        8666|
|Jammu and Kashmir|       10829|
|          Haryana|        8863|
|          Gujarat|       21371|
|           Sikkim|        5276|
|            Delhi|       22957|
|        Rajasthan|       22334|
|          Kerala |       13871|
|    